In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
# -*- coding: utf-8 -*-
"""
Author: ChatGPT
Idea:
1. 从 Kaggle Input 加载 universal_innovation_dataset.jsonl
2. 使用 BGE 中文向量模型编码 task
3. 构建 FAISS 余弦相似度索引（IndexFlatIP）
4. 保存索引与元数据到 /kaggle/working
5. 提供基础检索接口（非交互，适合 Kaggle）
"""

import os
import json
import numpy as np
import faiss
import torch
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

# ============================================================
# Kaggle 路径配置（⚠️ Input 只读，Output 写 working）
# ============================================================
JSONL_PATH = "/kaggle/input/universal-innovation-training-set/kaggle_upload/universal_innovation_dataset.jsonl"

INDEX_PATH = "/kaggle/working/innovation_faiss.index"
METADATA_PATH = "/kaggle/working/innovation_metadata.json"

# ============================================================
# 1. 加载 JSONL 数据
# ============================================================
def load_jsonl(filepath):
    texts = []
    metadata = []

    if not os.path.exists(filepath):
        raise FileNotFoundError(f"❌ 数据文件不存在: {filepath}")

    with open(filepath, "r", encoding="utf-8") as f:
        for line_num, line in enumerate(f, 1):
            line = line.strip()
            if not line:
                continue
            try:
                record = json.loads(line)
                task = record.get("task") or record.get("query_text")
                if not task:
                    continue
                texts.append(task)
                metadata.append(record)
            except Exception:
                continue

    print(f"✅ 成功加载 {len(texts):,} 条有效样本")
    return texts, metadata


# ============================================================
# 2. 构建 FAISS 索引
# ============================================================
def build_faiss_index(texts, index_path):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"🚀 使用设备: {device}")

    print("📥 加载 BGE 中文向量模型...")
    model = SentenceTransformer("BAAI/bge-large-zh-v1.5", device=device)

    print(f"🔄 编码文本向量（共 {len(texts):,} 条）...")
    vectors = model.encode(
        texts,
        batch_size=32,
        normalize_embeddings=True,
        show_progress_bar=True,
    )

    vectors = np.asarray(vectors, dtype="float32")
    dim = vectors.shape[1]

    print(f"📊 向量维度: {dim}")
    index = faiss.IndexFlatIP(dim)
    index.add(vectors)

    print(f"✅ FAISS 索引构建完成，共 {index.ntotal:,} 条")

    faiss.write_index(index, index_path)
    print(f"💾 索引已保存至: {index_path}")

    return index, model


# ============================================================
# 3. 保存 / 加载元数据
# ============================================================
def save_metadata(metadata, filepath):
    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(metadata, f, ensure_ascii=False)
    print(f"💾 元数据已保存至: {filepath}")


def load_metadata(filepath):
    if not os.path.exists(filepath):
        return None
    with open(filepath, "r", encoding="utf-8") as f:
        return json.load(f)


# ============================================================
# 4. 示例检索函数（适合 Kaggle，不用 input）
# ============================================================
def search_demo(model, index, metadata, query, top_k=3):
    query_vec = model.encode(
        [query],
        normalize_embeddings=True
    ).astype("float32")

    D, I = index.search(query_vec, top_k)

    print(f"\n🔍 Query: {query}")
    for rank, (idx, score) in enumerate(zip(I[0], D[0]), 1):
        item = metadata[idx]
        print(f"\n--- Top {rank} | 相似度 {score:.4f} ---")
        print("Task:", item.get("task") or item.get("query_text"))
        print("FOP:", item.get("fop", {}))
        print("IKS_ADV:", item.get("iks", {}).get("adv", []))


# ============================================================
# 5. 主流程（Kaggle 可直接运行）
# ============================================================
if __name__ == "__main__":
    print("=" * 60)
    print("🚀 Neural TRIZ | FAISS 向量索引构建（Kaggle）")
    print("=" * 60)

    texts, metadata = load_jsonl(JSONL_PATH)

    if not texts:
        raise RuntimeError("❌ 未加载到任何有效 task")

    if os.path.exists(INDEX_PATH) and os.path.exists(METADATA_PATH):
        print("📌 发现已有索引，直接加载")
        index = faiss.read_index(INDEX_PATH)
        metadata = load_metadata(METADATA_PATH)

        device = "cuda" if torch.cuda.is_available() else "cpu"
        model = SentenceTransformer("BAAI/bge-large-zh-v1.5", device=device)
    else:
        print("🆕 未发现索引，开始构建")
        index, model = build_faiss_index(texts, INDEX_PATH)
        save_metadata(metadata, METADATA_PATH)

    # =========================
    # 检索测试（示例）
    # =========================
    test_queries = [
        "降低电子设备的散热温度",
        "减少机械结构的振动和噪声",
        "提高太阳能电池转换效率"
    ]

    for q in test_queries:
        search_demo(model, index, metadata, q, top_k=3)


2026-02-03 07:31:45.661901: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770103905.862114      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770103905.919253      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770103906.408751      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770103906.408798      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770103906.408801      55 computation_placer.cc:177] computation placer alr

🚀 Neural TRIZ | FAISS 向量索引构建（Kaggle）
✅ 成功加载 124,532 条有效样本
🆕 未发现索引，开始构建
🚀 使用设备: cuda
📥 加载 BGE 中文向量模型...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

🔄 编码文本向量（共 124,532 条）...


Batches:   0%|          | 0/3892 [00:00<?, ?it/s]

📊 向量维度: 1024
✅ FAISS 索引构建完成，共 124,532 条
💾 索引已保存至: /kaggle/working/innovation_faiss.index
💾 元数据已保存至: /kaggle/working/innovation_metadata.json

🔍 Query: 降低电子设备的散热温度

--- Top 1 | 相似度 0.8924 ---
Task: 能够提高电子设备的散热性能
FOP: {'function': ['处理器', '夹角', '主体', '设置', '获取', '电子设备', '折叠', '指令', '检测', '改变', '模块', '执行', '距离', '控制', '温度'], 'object': ['所述电子设备', '电子设备', '第二温度模块', '第一温度模块', '第一获取模块', '控制模块'], 'process': ['设置', '电子设备', '检测', '第一温度值', '第二温度值', '获取', '第一温度模块', '充电状态的情况下，获取所述第一温度模块检测的第一温度', '第一获取模块', '充电状态', '充电', '折叠', '第二温度模块', '温度', '控制模块', '控制装置', '控制方法', '包括', '装置']}
IKS_ADV: ['克里斯坦森效应', '冷却', '荧光', '散射', '第二声音', '（万有）引力', '热释光法', '负折射现象', '蠕变波(爬行波)', '法拉第效应（磁旋转）', '核聚变', '电介质加热', '热霍尔效应', '热（分）解（散热）', '红外辐射', '光致发光', '爆燃', '辐射发光', '阴影', '布拉格衍射']

--- Top 2 | 相似度 0.8273 ---
Task: 提高集成电路设备的散热性
FOP: {'function': ['若干', '底座', '设置', '引线', '转轴', '电机', '测量', '顶部', '扇片', '通线', '散热', '齿轮', '涂层', '电子', '空腔', '凹槽'], 'object': ['空腔', '底部', '凹槽', '电机', '第二转轴', '封装层', '散热块', '扇片', '散热孔', '设备主体', '第一转轴

In [2]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split

# =========================
# 配置
# =========================
INPUT_PATH = r"/kaggle/input/universal-innovation-training-set/kaggle_upload/universal_innovation_dataset.jsonl"
OUTPUT_CSV = r"/kaggle/working/t5_innovation_train.csv"

data_pairs = []

print("📖 正在构建 T5 训练语料...")
with open(INPUT_PATH, "r", encoding="utf-8") as f:
    for line in f:
        try:
            item = json.loads(line)
            
            # 1. 构建 Input (模拟 TRIZ 的矛盾输入)
            # 格式：Problem: [任务] | Improve: [属性] | Avoid: [负面影响]
            task = item.get("task", "")
            
            # 提取矛盾属性 (ADJ)
            adj_info = item.get("adj", {})
            direction = adj_info.get("direction", "none")
            attrs = ",".join(adj_info.get("attribute", []))
            
            input_prompt = f"Innovation Task: {task}"
            if attrs:
                if direction == "increase":
                    input_prompt += f" | Goal: Increase {attrs}"
                elif direction == "decrease":
                    input_prompt += f" | Goal: Reduce {attrs}"
                else:
                    input_prompt += f" | Focus: {attrs}"
            
            # 2. 构建 Target (纯粹的创新方法“种子”)
            # 格式：Method: [功能动作] using [科学效应]
            fops = ",".join(item.get("fop", {}).get("function", [])[:5]) # 取核心动作
            advs = ",".join(item.get("iks", {}).get("adv", [])[:5])       # 取核心效应
            
            # 如果没有效应，只用动作；如果都有，组合之
            if advs:
                target_text = f"Action: {fops} | Principle: {advs}"
            elif fops:
                target_text = f"Action: {fops}"
            else:
                continue # 没有方法的数据丢弃
                
            data_pairs.append([input_prompt, target_text])
            
        except Exception:
            continue

# 转为 DataFrame 并保存
df = pd.DataFrame(data_pairs, columns=["input_text", "target_text"])
print(f"✅ 构建完成，共 {len(df)} 条训练样本。")
print("示例 Input: ", df.iloc[0]["input_text"])
print("示例 Target:", df.iloc[0]["target_text"])

# 划分训练集和验证集
train_df, val_df = train_test_split(df, test_size=0.05, random_state=42)
train_df.to_csv(r"/kaggle/working/t5_train.csv", index=False)
val_df.to_csv(r"/kaggle/working/t5_val.csv", index=False)

📖 正在构建 T5 训练语料...
✅ 构建完成，共 124283 条训练样本。
示例 Input:  Innovation Task: 为所述匿名用户生成唯一ID并记录访问来源 通过在用户首次匿名访问企业网站时生成唯一ID以保存后续用户在每次浏览交互过程中产生的交互数据并记录访问来源 实现对企业网站数据的全面准确追踪与分析 为衡量营销策略提供准确的分析数据 以提升用户转化率 | Focus: 访问,保存,检测,分组标签,匿名,采集
示例 Target: Action: 生成,试用,存储器,访问,记录


In [3]:
# ============================================================
# Author: ChatGPT + Grok Optimized (Kaggle Stability Edition)
# Purpose: Fine-tune T5 for Innovation Generation
# Fixes: DataLoader Deadlock, No Output, Training Hangs
# ============================================================

import os
import pandas as pd
import torch
import shutil
from torch.utils.data import Dataset
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq,
)

# 1. 强制清理显存 & 检查环境
torch.cuda.empty_cache()
print(f"🔥 PyTorch Version: {torch.__version__}")
print(f"🔥 CUDA Available: {torch.cuda.is_available()}")

# ============================================================
# 2. 配置与路径
# ============================================================
# ⚠️ 请确认你的 csv 文件确实在这个路径下
TRAIN_PATH = "/kaggle/working/t5_train.csv"
VAL_PATH   = "/kaggle/working/t5_val.csv"

# 检查文件是否存在，防止空跑
if not os.path.exists(TRAIN_PATH):
    raise FileNotFoundError(f"❌ 找不到训练数据: {TRAIN_PATH}，请检查上一步是否成功生成 CSV！")

MODEL_NAME = "Langboat/mengzi-t5-base"
OUTPUT_DIR = "/kaggle/working/t5_innovation_model"
FINAL_SAVE_PATH = "/kaggle/working/final_model"

# 稳健参数配置
BATCH_SIZE = 12          # 显存够可改 16，求稳用 8
EPOCHS = 3
LR = 2e-4
MAX_INPUT_LEN = 128
MAX_TARGET_LEN = 64

# ============================================================
# 3. 数据集类
# ============================================================
class InnovationT5Dataset(Dataset):
    def __init__(self, csv_path, tokenizer):
        # fillna("") 防止数据中有空值导致报错
        self.data = pd.read_csv(csv_path).fillna("")
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        
        # 强制转字符串，防止 float/int 混入
        input_text = str(row["input_text"])
        target_text = str(row["target_text"])

        inputs = self.tokenizer(
            input_text,
            max_length=MAX_INPUT_LEN,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        with self.tokenizer.as_target_tokenizer():
            labels = self.tokenizer(
                target_text,
                max_length=MAX_TARGET_LEN,
                padding="max_length",
                truncation=True,
                return_tensors="pt",
            )["input_ids"]

        # 忽略 padding 的 loss 计算
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "labels": labels.squeeze(),
        }

# ============================================================
# 4. 模型初始化
# ============================================================
print(f"🚀 Loading Model: {MODEL_NAME}")
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
model.to("cuda")

train_dataset = InnovationT5Dataset(TRAIN_PATH, tokenizer)
val_dataset = InnovationT5Dataset(VAL_PATH, tokenizer)
print(f"📊 Train Samples: {len(train_dataset)} | Val Samples: {len(val_dataset)}")

# ============================================================
# 5. 训练参数 (防死锁核心配置)
# ============================================================
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    
    # 训练超参
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LR,
    weight_decay=0.01,
    fp16=True,  # 开启混合精度加速
    
    # ✅ 稳定性修改 1: 按 Step 保存和评估，避免 Epoch 结束时的 IO 洪峰
    save_strategy="steps",
    save_steps=500,        # 每 500 步存一次
    eval_strategy="steps", # 新版 transformers 参数
    eval_steps=500,
    
    # ✅ 稳定性修改 2: 限制保存数量，防止磁盘写满
    save_total_limit=2,
    load_best_model_at_end=True, # 只有正常结束才加载最优模型
    
    # ✅ 稳定性修改 3: 必须设为 0！解决 Kaggle 死锁的唯一解
    dataloader_num_workers=0, 
    
    logging_steps=100,
    report_to="none", # 关闭 wandb 减少联网干扰
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
)

# ============================================================
# 6. 训练执行 (带保险丝)
# ============================================================
print("\n🔥 Starting Training with Deadlock Protection...")

try:
    trainer.train()
    print("✅ Training finished successfully.")
except Exception as e:
    print(f"\n⚠️ Training interrupted due to error: {e}")
    print("⚠️ Attempting to save current state...")

# ============================================================
# 7. 强制保存 (无论成败)
# ============================================================
print(f"\n💾 Saving final model to: {FINAL_SAVE_PATH}")
# 确保目录存在
os.makedirs(FINAL_SAVE_PATH, exist_ok=True)

# 保存模型
trainer.save_model(FINAL_SAVE_PATH)
tokenizer.save_pretrained(FINAL_SAVE_PATH)

# 验证保存结果
if os.path.exists(os.path.join(FINAL_SAVE_PATH, "config.json")):
    print("✅ Model saved successfully! You can find it in the output directory.")
else:
    print("❌ Save might have failed, checking checkpoints...")
    if os.path.exists(OUTPUT_DIR):
        print(f"📂 Checkpoints found: {os.listdir(OUTPUT_DIR)}")

# ============================================================
# 8. 推理测试 (验证模型是否变聪明了)
# ============================================================
print("\n🧪 Running Inference Tests...")
model.eval()

def generate_solution(task):
    input_text = f"Innovation Task: {task}"
    input_ids = tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True).input_ids.to("cuda")
    
    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            max_length=64,
            num_beams=5,
            early_stopping=True,
            no_repeat_ngram_size=2 # 防止复读机
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

test_cases = [
    "提高太阳能电池板的转换效率 | Goal: Increase 光电转化率",
    "减少齿轮传动过程中的噪音 | Goal: Reduce 振动",
    "解决手机在高负荷下的发热问题 | Goal: Reduce 温度"
]

for case in test_cases:
    print(f"\n🔹 Input: {case}")
    print(f"🔸 Output: {generate_solution(case)}")

2026-02-04 02:42:17.625606: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770172937.807510      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770172937.862726      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770172938.314813      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770172938.314856      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770172938.314859      55 computation_placer.cc:177] computation placer alr

🔥 PyTorch Version: 2.8.0+cu126
🔥 CUDA Available: True
🚀 Loading Model: Langboat/mengzi-t5-base


spiece.model:   0%|          | 0.00/725k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

📊 Train Samples: 118068 | Val Samples: 6215

🔥 Starting Training with Deadlock Protection...


/tmp/ipykernel_55/825679522.py:136: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/data/data_collator.py:740: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int

Step,Training Loss,Validation Loss
500,1.123600,1.033239
1000,1.006400,0.967538
1500,0.968400,0.936144
2000,0.940100,0.913750
2500,0.929000,0.895926
3000,0.896900,0.883546
3500,0.923700,0.875228
4000,0.895000,0.860976
4500,0.881900,0.857711
5000,0.860700,0.848227


/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

✅ Training finished successfully.

💾 Saving final model to: /kaggle/working/final_model
✅ Model saved successfully! You can find it in the output directory.

🧪 Running Inference Tests...

🔹 Input: 提高太阳能电池板的转换效率 | Goal: Increase 光电转化率
🔸 Output: Action: | Principle: 光生伏打效应(光伏效应),阴极发光,光电导性,静电感应,欧杰效应

🔹 Input: 减少齿轮传动过程中的噪音 | Goal: Reduce 振动
🔸 Output: Action: | Principle: 拉胀材料,转矩(力矩),附加质量,达尔文漂移,速率比

🔹 Input: 解决手机在高负荷下的发热问题 | Goal: Reduce 温度
🔸 Output: Action: | Principle: 热离子(电子)发射,冷却,第二声音,光电导性,静电感应
